## load model

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq
last_checkpoint_34000 = "./third-results/checkpoint-4000"

tokenizer = T5Tokenizer.from_pretrained(last_checkpoint_34000)
model = T5ForConditionalGeneration.from_pretrained(last_checkpoint_34000, device_map="auto")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

c:\Users\User\miniconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## load dataset

In [2]:
from datasets import load_dataset
ds = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k")
ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 112165
    })
})

## split the dataset

In [3]:
ds=ds['train'].take(80000)
ds=ds.train_test_split(test_size=0.3)
ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 56000
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 24000
    })
})

## processing dataset

In [4]:
prefix = "If you are a doctor, please answer the medical questions based on the patient's description: "

# Define the preprocessing function

def preprocess_function(examples):
   inputs = [prefix + doc for doc in examples["input"]]
   model_inputs = tokenizer(inputs, max_length=512, truncation=True)
  
   labels = tokenizer(text_target=examples["output"], 
                      max_length=512,         
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

tokenized_dataset = ds.map(preprocess_function, batched=True)

Map: 100%|██████████| 24000/24000 [00:13<00:00, 1841.99 examples/s]


## compute_metrics

In [5]:
import nltk
import evaluate
import numpy as np

nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result

## fine-tuning

In [6]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./dataset2-results",
    evaluation_strategy="epoch",
    learning_rate=2e-4,  
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=4, 
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=5,
    predict_with_generate=True,
    push_to_hub=False
   
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()



c:\Users\User\miniconda3\envs\transformers\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 10/21000 [00:02<1:28:02,  3.97it/s]

{'loss': 4.1581, 'grad_norm': 1.4983924627304077, 'learning_rate': 0.0001999047619047619, 'epoch': 0.0}


  0%|          | 20/21000 [00:05<1:33:34,  3.74it/s]

{'loss': 3.9781, 'grad_norm': 1.1323121786117554, 'learning_rate': 0.00019980952380952382, 'epoch': 0.0}


  0%|          | 30/21000 [00:09<2:00:07,  2.91it/s]

...

100%|██████████| 21000/21000 [3:26:59<00:00,  3.07it/s]

{'loss': 2.4977, 'grad_norm': 1.1993515491485596, 'learning_rate': 0.0, 'epoch': 3.0}


c:\Users\User\miniconda3\envs\transformers\lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                                       
100%|██████████| 21000/21000 [3:55:02<00:00,  1.49it/s]

{'eval_loss': 2.47175669670105, 'eval_rouge1': 0.13476575672545743, 'eval_rouge2': 0.035938374575499546, 'eval_rougeL': 0.1050483027458741, 'eval_rougeLsum': 0.12841317326037144, 'eval_runtime': 1682.264, 'eval_samples_per_second': 14.266, 'eval_steps_per_second': 3.567, 'epoch': 3.0}
{'train_runtime': 14102.4473, 'train_samples_per_second': 11.913, 'train_steps_per_second': 1.489, 'train_loss': 2.7802899896530877, 'epoch': 3.0}


TrainOutput(global_step=21000, training_loss=2.7802899896530877, metrics={'train_runtime': 14102.4473, 'train_samples_per_second': 11.913, 'train_steps_per_second': 1.489, 'total_flos': 1.4212368909631488e+16, 'train_loss': 2.7802899896530877, 'epoch': 3.0})

## inference

In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
last_checkpoint = "./third-results/checkpoint-4000"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint).to("cuda")
finetuned_tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)
question="what are marine toxins?"

input_text = "Please answer this medical related question: "+question
input_ids = finetuned_tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = finetuned_model.generate(
    input_ids,
    max_length=200,
    min_length=20,
    repetition_penalty=2.0
)
answer = finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)
from textwrap import fill

print(fill(answer, width=100))

c:\Users\User\miniconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Marine toxins are substances that cause damage to the body's tissues and organs. They can be toxic,
but they do not affect other parts of the body. The most common types of marine toxins include:
Lymphadenoma (the type of lymph nodes in the blood) Affected people may have an increased risk for
developing certain diseases such as cancer or heart disease. Some cases of this condition occur when
there is too much fluid in the brain or spinal cord. In some instances, it causes pain, swelling,
loss of appetite, nausea, vomiting, diarrhea, headache, seizures, fatigue, weight gain, muscle
weakness, difficulty swallowing, and/or confusion.
